In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/tobias/Repositories/biaslyze/')

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

## Load and prepare data

In [4]:
df = pd.read_csv("../data/disaster-tweets/train.csv"); df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# replace urls
import re
url_regex = re.compile("(http|https)://[\w\-]+(\.[\w\-]+)+\S*")

df = df.replace(to_replace=url_regex, value='', regex=True)

## Train a model

In [6]:
clf = make_pipeline(TfidfVectorizer(min_df=10, max_features=10000, stop_words="english"), LogisticRegression())

In [7]:
clf.fit(df.text, df.target)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=10000, min_df=10,
                                 stop_words='english')),
                ('logisticregression', LogisticRegression())])

In [8]:
train_pred = clf.predict(df.text)
print(accuracy_score(df.target, train_pred))

0.8418494680152371


## Test detection of concepts

In [9]:
from biaslyze.concept_detectors import KeywordConceptDetector
from biaslyze.evaluators import LimeBiasEvaluator

/home/tobias/.cache/pypoetry/virtualenvs/biaslyze-OjqpgMw8-py3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
key_detect = KeywordConceptDetector()

In [11]:
detected_tweets = key_detect.detect(texts=df.text[:600])

In [12]:
len(detected_tweets)

72

In [13]:
detected_tweets

['13,000 people receive #wildfires evacuation orders in California ',
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 '#Flood in Bago Myanmar #We arrived Bago',
 'How the West was burned: Thousands of wildfires ablaze in California alone ',
 'How the West was burned: Thousands of wildfires ablaze in #California alone  #climate #energy ',
 "@Navista7 Steve these fires out here are something else! California is a tinderbox - and this clown was setting my 'hood ablaze @News24680",
 ' Twelve feared killed in Pakistani air ambulance helicopter crash ',
 'Twelve feared killed in Pakistani air ambulance helicopter crash - Reuters  #yugvani',
 'Twelve feared killed in Pakistani air ambulance helicopter crash ',
 '#news Twelve feared killed in Pakistani air ambulance helicopter crash  #til_now #DNA',
 ' Twelve feared killed in Pakistani air ambulance helicopter crash ',
 'Pakistan air ambulance helicopter crash kills nine ',
 '

## Test LIME based bias detection with keywords

In [27]:
from biaslyze.bias_detectors import LimeKeywordBiasDetector

In [80]:
bias_detector = LimeKeywordBiasDetector(bias_evaluator=LimeBiasEvaluator(n_lime_samples=500), n_top_keywords=10, use_tokenizer=True)

In [81]:
detection_res = bias_detector.detect(texts=df.text[500:600], predict_func=clf.predict_proba)

2023-03-07 09:51:49.907 | INFO     | biaslyze.concept_detectors:detect:32 - Started keyword-based concept detection on 100 texts...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 86.63it/s]
2023-03-07 09:51:51.067 | INFO     | biaslyze.concept_detectors:detect:46 - Done. Found 15 texts with protected concepts.
2023-03-07 09:51:51.068 | INFO     | biaslyze.evaluators:evaluate:40 - Started bias detection on 15 samples...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:03<00:00,  4.23it/s]


In [82]:
detection_res.summary()

Detected 11 samples with potential issues.
    Potentially problematic concepts detected: [('nationality', 11), ('religion', 10)]
    Based on keywords: [('israeli', 10), ('christian', 8), ('muslims', 8), ('australia', 1)].


In [83]:
detection_res.details(group_by_concept=True)

Concept: nationality
[{'reason': ['israeli', 'christian'],
  'text': 'Christian Attacked by Muslims at the Temple Mount after Waving '
          'Israeli Flag via Pamela Geller - ... '},
 {'reason': ['israeli', 'muslims'],
  'text': 'Christian Attacked by Muslims at the Temple Mount after Waving '
          'Israeli Flag via Pamela Geller - ... '},
 {'reason': ['israeli', 'muslims', 'christian'],
  'text': 'Christian Attacked by Muslims at the Temple Mount after Waving '
          'Israeli Flag via Pamela Geller - ... '},
 {'reason': ['israeli', 'muslims'],
  'text': 'Christian Attacked by Muslims at the Temple Mount after Waving '
          'Israeli Flag via Pamela Geller - ... '},
 {'reason': ['israeli', 'muslims', 'christian'],
  'text': 'Christian Attacked by Muslims at the Temple Mount after Waving '
          'Israeli Flag via Pamela Geller - ... '},
 {'reason': ['israeli', 'muslims', 'christian'],
  'text': 'Christian Attacked by Muslims at the Temple Mount after Waving '
      

## Test masked language model based bias detection with keywords

In [19]:
from biaslyze.bias_detectors import MaskedKeywordBiasDetector

In [20]:
bias_detector = MaskedKeywordBiasDetector(predict_func=clf.predict_proba, n_resample_keywords=15)

In [24]:
detection_res = bias_detector.detect(texts=df.text[500:600])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:23<00:00,  1.80s/it]


In [25]:
detection_res

Detected 1 samples with potential issues.
    Potentially problematic concepts detected: Counter({'nationality': 1})
    Based on keywords: Counter({'german': 1}).

In [26]:
detection_res.details()

''A young German stormtrooper engaged in the Battle of the Somme 1916. [800 ÌÑ 582 ] '' might contain bias ['nationality']; reasons: ['german']
